## Cargar `encoder`

In [1]:
import numpy as np
aux = np.random.rand(1,128,128,1) # (batch, height, width, channel)

In [2]:
import tensorflow as tf
from unsupervised_dna import ModelLoader, LoadImageEncoder

2021-09-14 14:56:18.615396: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-14 14:56:18.615435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
loader = ModelLoader()

In [4]:
vae = loader("vae_7mer", weights_path = "checkpoint/cp.ckpt")

encoder = tf.keras.Model(
                   inputs=vae.input,
                   outputs=vae.layers[1].output
        )

2021-09-14 14:56:23.468491: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-14 14:56:23.468532: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-14 14:56:23.468557: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Koke): /proc/driver/nvidia/version does not exist
2021-09-14 14:56:23.468767: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 **load model weights_path** : checkpoint/cp.ckpt

**Model created**


In [5]:
#[(layer.output, layer) for layer in model.layers]  

### random example

In [6]:
output = encoder(aux)
z_mean, z_var, z_sample = output

In [7]:
z_mean.numpy()

array([[-0.00076576,  0.00388247, -0.0055711 ]], dtype=float32)

### img example

In [8]:
encoder_img = LoadImageEncoder(128,128)

In [9]:
img = encoder_img(file_path="data/fcgr-7-mer/00000-hCoV-19_Australia_NT12_2020|2020-03-25|2020-04-17.jpg")

In [10]:
encoder(tf.expand_dims(img/255., axis=0, name="batch"))

[<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 6.4424047e-04,  5.3554035e-05, -1.1291959e-04]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.00475141, -0.00127108, -0.00178111]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.47035474, -1.2880545 ,  1.3038085 ]], dtype=float32)>]

### with tf.data API example
___

In [14]:
from unsupervised_dna import (
    LoadImageEncoder,
    DatasetEncoder,
    MonitorValues,
)

In [15]:
ds_encoder = DatasetEncoder(data_dir="data/fcgr-7-mer", batch_size=1, kmer=7)

dataset loaded from: data/fcgr-7-mer | Total files: 27514


In [13]:
ds_test = ds_encoder()
for f in ds_test.ds.take(5):
    print(f.numpy())

AttributeError: 'MapDataset' object has no attribute 'ds'

In [ ]:
iter_test = iter(ds_test)

In [ ]:
img = next(iter_test)

In [ ]:
encoder(img)

In [16]:
ds_encoder.ds

<ShuffleDataset shapes: (), types: tf.string>

In [17]:
from pathlib import Path
DATA_DIR = Path("data/fcgr-7-mer") # image directory
LIST_IMG = DATA_DIR.rglob("*.jpg")
